# **Chapter 9. Complex Prompt Chaining**

Prompt chaining is a technique in prompt engineering where multiple prompts are linked together to guide the model through a series of logical steps or stages to complete a complex task. This technique is particularly **useful for breaking down complex tasks into smaller, more manageable parts.** The output of one prompt serves as the input for the next, creating a chain that leads to the final answer.

This method is especially useful in scenarios where a single prompt isn't sufficient to solve the problem, such as providing accurate responses in customer support chatbots or writing research papers. 

Here are some examples of prompt chaining examples across different domains:

**1. Legal and Business Analysis** 

---

- **Contract Review**:  

    - Identify clauses → Assess risks → Recommend changes → Summarize key points.  

- **Due Diligence**:

    - Gather financial data → Assess liabilities → Verify legal compliance → Generate a risk report.  

**2. Customer Support Resolution**

---

- **Query Handling**:  

    - Classify issue → Retrieve relevant information → Generate initial response → Adjust for tone → Confirm solution accuracy.  

- **Feedback Processing**:  

    - Collect feedback → Categorize themes → Analyze sentiment → Recommend actionable improvements.  


**3. Data Science** 

---

- **Predictive Modeling**:  

    - Clean data → Feature selection  → Evaluate → Refine → Deploy.  

- **Customer Segmentation**:  

    - Data extraction → Segmentation analysis → Validate clusters → Profile each segment → Recommend targeting strategies.  

**4. Academic Research** 

---

- **Research and Review**:  

    - Find relevant sources → Summarize main findings → Identify research gaps.  

- **Analyze and Interpret**:  

    - Collect and analyze data → Interpret results → Formulate key conclusions.  

- **Draft and Refine**:  

    - Write main sections (intro, methods, results) → Edit for clarity and coherence → Finalize and format.  

**Set up**

In [ ]:
from openai import OpenAI

# Retrieve the UPSTAGE_API_KEY variable from the IPython store
%store -r UPSTAGE_API_KEY

try:
    if UPSTAGE_API_KEY:
        print("Success!")
except NameError as ne:
    print(f"Since, {ne}")
    print("Please, insert your API key.")
    UPSTAGE_API_KEY = input("UPSTAGE_API_KEY =")

# Set your API key: 
# UPSTAGE_API_KEY = " " ←- Insert your API key here. 

client = OpenAI(
    api_key= UPSTAGE_API_KEY,
    base_url="https://api.upstage.ai/v1/solar"
)

config_model = {
    "model": "solar-pro",
    "max_tokens": 996,
    "temperature": 0.5,
    "top_p": 1.0,
}

def get_completion(messages, system_prompt="", config=config_model):
    try:
        if system_prompt:
            messages = [{"role": "system", "content": system_prompt}] + messages

        message = client.chat.completions.create(messages=messages, **config)
        return message.choices[0].message.content
    
    except Exception as e:
        print(f"Error during API call: {e}")
        return None

Success!


---

## **9.1 Create Contextually Enriched Steps**

Imagine you're building a customer response chatbot to handle frequently asked inquiries.
A single prompt might look like this:

*System Prompt*: Build a customer response chatbot and provide a response to the following customer query.
*User Prompt*: What is your return policy?

*Output*:
"Our return policy allows you to return most items within 30 days of delivery for a full refund or exchange. Please note that some items, such as personal care products and final sale items, are not eligible for return. For more information, please visit our website's return policy page or contact our customer support team."

While this response seems fine, it is actually false information generated by Solar. To address this issue, we will use some fake FAQs generated by Solar.

In [ ]:
FAQ = """
Company name: GreenPulse Innovations. 

1. **How can I reset my password?**
    - To reset your password, go to the login page and click on "Forgot Password." Enter your registered email address, and we’ll send you instructions to reset your password.
2. **Where can I find my order status?**
    - You can check your order status by logging into your account and navigating to the "Order History" section. You’ll find the current status and tracking information there.
3. **What is your return policy?**
    - We offer a 30-day return policy for most items. To return an item, ensure it’s in its original condition and packaging, and submit a return request via your account.
4. **How do I contact customer support?**
    - You can reach our customer support team by sending an email to support@fakestore.com or using the live chat feature on our website, available Monday to Friday from 9 AM to 5 PM.
5. **Do you offer international shipping?**
    - Yes, we ship to most countries worldwide. Shipping fees and delivery times vary depending on your location. Please check the shipping section on our website for more details.
6. **How do I cancel my subscription?**
    - To cancel your subscription, go to your account settings and select the "Subscriptions" tab. You will find the option to cancel your current plan there.
7. **Can I change my billing information?**
    - Yes, you can update your billing information by going to the "Account Settings" page. Click on "Billing Information," and you can edit your payment method or billing address.
8. **Why was my payment declined?**
    - Payments may be declined due to incorrect payment information, insufficient funds, or issues with your card provider. Please double-check your details and try again. If the issue persists, contact your payment provider.
9. **What should I do if I received a damaged product?**
    - If you received a damaged product, please contact customer support with a photo of the damage. We’ll assist you with a replacement or refund.
10. **How can I update my shipping address?**
    - You can update your shipping address by logging into your account and going to the "Address Book" section. From there, you can add or modify your address.
"""

The prompt we are going to craft consists of three steps. 
- **Approach**: First, identify the user's intent through an intent-detection step. Then, refine the query with follow-up prompts to ensure clarity. We will create a chatbot that answers user questions by including a list of frequently asked questions (FAQs) in the prompt. In this case, prompt chaining is highly beneficial.

In [ ]:
# Step 1: Understanding the Issue

## Initial prompt
first_prompt = """Extract the main issue from the user's query and respond with a phrase.
The Main Issue: {}"""

## Define the User Query
user_prompt = "How do I reset my password?"

message = [
    {
        "role": "system",
        "content": first_prompt
    },
    {
        "role": "user",
        "content": user_prompt
    }
]

first_response = get_completion(message)
print(first_response, "\n\n")

The Main Issue: Password Reset 




In [ ]:
# Step 2: FAQ Lookup 
second_prompt = """Find the user query in the <FAQ>. 
If not, say,"I am sorry, please contact a person via email"
Here is an FAQ for you to reference when you answer.  

################################################
<FAQ> 
{FAQ}
</FAQ> 

################################################
- user query: {user_query}
- Question Number: [ ]
- And its answer:  """

message = [
    {
        "role": "user",
        "content": first_response
    },
    {
        "role": "system",
        "content": second_prompt.format(FAQ=FAQ, user_query=user_prompt)
    }
]

second_response = get_completion(messages=message)
print(second_response, "\n\n")

Question Number: [1]
And its answer: To reset your password, go to the login page and click on "Forgot Password." Enter your registered email address, and we’ll send you instructions to reset your password. 




In [ ]:
# Step 3: Final Response
third_prompt = """Provide an answer to the {user_query}. 
When responding, observe the following interactional rule. 
1. Only answer questions that are covered in <FAQ>. 
2. If user asks about something outside of <FAQ>, please say 'I am here to solve your question about the services.'
3. Be polite and kind. 

- user query: <user_query> 
- Your final answer(But don't include this phrase): """

message = [
    {
        "role": "user",
        "content": second_response
    },
    {
        "role": "system",
        "content": third_prompt.replace("<user_query>", user_prompt)
    }
]

final_response = get_completion(messages=message)
print(final_response, "\n\n")

To reset your password, go to the login page and click on "Forgot Password." Enter your registered email address, and we’ll send you instructions to reset your password. 




---

## **9.2 Address a Specific Phase of the Workflow**

You can chain prompts to have Solar perform research tasks. Now, we will practice using prompt chaining with the main sections, starting with a literature reiveiw. 

This workflow involves the following steps:  

 (1) **summarizing based on the given text sources.**  

 (2) **editing for clarity and coherence.**  
 
 (3) **finalizing the output in a specified format (e.g., APA).**  

To complete this task, we are using the book titled *Decoding Korean Political Talk: From Data to Debate* ([S. Kang, 2024, Routledge: London](https://www.routledge.com/Decoding-Korean-Political-Talk-From-Data-to-Debate/Kang/p/book/9781032737263?srsltid=AfmBOor_uhsxjrMICoZscejpDSmphdRJZTdlVgyGUcD1xqYst2zHUMjA)). We will use a portion of the book as practice material for prompt chaining exercises.

**Text Source**: (p. 10)

In [ ]:
text = """
*Prior CA research has extensively studied question-answer sequences across*
*languages, revealing their central functions in interactive talk, with most studies*
*applying these findings within pedagogical contexts. However, studies*
*examining these sequences in institutional settings are less common.*
*Heritage’s (2002b) research on institutional interactions found that participants’*
*conduct and way of speaking often reflects the norms and constraints*
*of the situation. For example, how lawyers phrase questions in courtrooms*
*will differ from everyday conversation due to institutional rules and roles. In*
*such formal settings, there are prescribed procedures and expectations that*
*influence linguistic behavior.6 As a result, the formulation of questions and*
*responses in these contexts is significantly influenced by the institutional setting,*
*leading to distinct linguistic patterns and strategies employed by participants*
*to navigate the specific communicative demands of the situation.*
*Clayman and Heritage’s study (2002b) on televised news interviews with*
*political figures provides particularly relevant insights into the dynamics of*
*such interactions. Through conversation analysis, they meticulously dissected*
*the structure and content of these interviews, revealing a pattern where interviewers*
*often embed implicit assumptions or agendas within their questions.*
*For instance, the use of a negative interrogative like “Don’t you think you*
*misled voters?” subtly expresses the interviewer’s presumption that the interviewee*
*did mislead voters. This tactic adds a layer of challenge to the question*
*compared to typical everyday conversation, as it not only seeks information*
*but also implies a certain judgement.*
"""

In [ ]:
# Step 1. Initial Summary

initial_prompt = f"""
Read the provided text and summarize its main points based on improtant keywords:
####
<Text>
{text}
</Text>
"""

message = [
    {
        "role": "user",
        "content": initial_prompt
    }
]

initial_summary = get_completion(messages=message)

# Print with formatting
print("\n=== Summary ===")
print("-" * 80)
print(initial_summary)
print("-" * 80)
print("\n\n")


=== Summary ===
--------------------------------------------------------------------------------
The text discusses the impact of institutional settings on question-answer sequences, highlighting that participants' conduct and language are influenced by the norms and constraints of the situation. The text cites Heritage's (2002b) research on institutional interactions and Clayman and Heritage's (2002b) study on televised news interviews with political figures. The findings reveal that in formal settings, there are prescribed procedures and expectations that influence linguistic behavior. For example, interviewers in televised news interviews often embed implicit assumptions or agendas within their questions, such as using negative interrogatives to imply a certain judgement. The text emphasizes that the formulation of questions and responses in institutional settings is significantly influenced by the specific communicative demands of the situation, leading to distinct linguistic patt

In [ ]:
# Step 2: Editing for Clarity and Coherence

clarity_prompt = """1. Review the summary result. Edit for clarity and coherence,ensuring the language is concise and logically flows. 
2. Then, rewrite any complex sentences to make them easier to understand while keeping the original meaning. 
3. Follow response format: 
- Revised sections:
- Reasons for revision: 

<Summary>
{summary}
</Summary>
"""

message = [
    {
        "role": "user",
        "content": clarity_prompt.format(summary=initial_summary)
    }
]

edited_version = get_completion(messages=message)

# formatting 
print("\n=== Edited Version ===")
print("-" * 80)
print(edited_version)
print("-" * 80)
print("\n\n")


=== Edited Version ===
--------------------------------------------------------------------------------
Revised sections:

1. The text discusses how institutional settings affect question-answer sequences, emphasizing that participants' behavior and language are shaped by the norms and restrictions of the situation.
2. The text refers to Heritage's (2002b) research on institutional interactions and Clayman and Heritage's (2002b) study on televised news interviews with political figures.
3. The findings reveal that in formal settings, there are set procedures and expectations that influence how people use language.
4. For instance, interviewers in televised news interviews often include hidden assumptions or agendas in their questions, such as using negative interrogatives to suggest a particular judgement.
5. The text highlights that the way questions and answers are formed in institutional settings is greatly influenced by the specific communication needs of the situation, resulting 

In [ ]:
# Step 3: Finalizing in APA Format
final_prompt = f"""Please prepare it for final submission. 
Extract all references and list them in APA format.
####
<Text>
{edited_version}
</Text>
"""

message = [
    {
        "role": "user",
        "content": final_prompt
    }
]

APA_ref = get_completion(messages=message)

print(APA_ref, "\n\n")

Here are the references listed in APA format:

1. Clayman, S. I., & Heritage, J. (2002b). Institutional interactions: Television news interviews with political figures. In J. M. Schachter & B. A. Held (Eds.), Handbook of discourse analysis (pp. 173-196). Blackwell Publishing.
2. Heritage, J. (2002b). Institutional interactions: Television news interviews with political figures. In J. M. Schachter & B. A. Held (Eds.), Handbook of discourse analysis (pp. 173-196). Blackwell Publishing. 




In [ ]:
# Summarization 
print("\n=== Initial Summary ===")
print("-" * 80)
print(initial_summary)
print("-" * 80)
print("\n=== Edited Version ===")
print("-" * 80)
print(edited_version)
print("\n=== Reference ===")
print(APA_ref)
print("\n\n")


=== Initial Summary ===
--------------------------------------------------------------------------------
The text discusses the impact of institutional settings on question-answer sequences, highlighting that participants' conduct and language are influenced by the norms and constraints of the situation. The text cites Heritage's (2002b) research on institutional interactions and Clayman and Heritage's (2002b) study on televised news interviews with political figures. The findings reveal that in formal settings, there are prescribed procedures and expectations that influence linguistic behavior. For example, interviewers in televised news interviews often embed implicit assumptions or agendas within their questions, such as using negative interrogatives to imply a certain judgement. The text emphasizes that the formulation of questions and responses in institutional settings is significantly influenced by the specific communicative demands of the situation, leading to distinct linguis

**References** 

Kang, S. (2024). *Decoding Korean Political Talk: From Data to Debate.*  Taylor & Francis.
